In [124]:
grid = []
with open('day_15_input') as f:
    for line in f.readlines():
        grid.append([int(s) for s in line.strip()])

In [125]:
class Node(object):
    
    def __init__(self, idx, value):
        self.neighbors = []
        self.idx = idx
        self.value = value
        self.end = False
                
nodes = []
        
width = len(grid[0])
height = len(grid)
    
def add_neighbor(nodes, idx, other_idx):
    nodes[idx].neighbors.append(nodes[other_idx])
    nodes[other_idx].neighbors.append(nodes[idx])
    
for row in range(height):
    for col in range(width):
        node = Node(len(nodes), grid[row][col])
        idx = len(nodes)
        nodes.append(node)
        if row > 0:
            # if col > 0:
            #     add_neighbor(nodes, idx, idx - width - 1)
            add_neighbor(nodes, idx, idx - width)
            # if col < width - 1:
            #     add_neighbor(nodes, idx, idx - width + 1)
        if col > 0:
            add_neighbor(nodes, idx, idx - 1)
            
nodes[-1].end = True

In [126]:
big_nodes = [None] * len(nodes) * 5 * 5
def new_cost(idx, grid_num):
    old_cost = nodes[idx].value
    next_cost = old_cost + grid_num
    if next_cost > 9:
        next_cost = next_cost - 9
    return next_cost

for y_idx in range(5):
    for x_idx in range(5):
        for i in range(len(nodes)):
            x = i % width
            y = i // width
            new_idx = x_idx * width + x + y_idx * (height * width * 5) + (y * width * 5)
            node = Node(new_idx, new_cost(i, y_idx + x_idx))
            big_nodes[new_idx] = node
            
print(len(big_nodes))
fullwidth = width * 5
fullheight = height * 5
for row in range(fullheight):
    for col in range(fullwidth):
        idx = row * fullwidth + col
        if row > 0:
            add_neighbor(big_nodes, idx, idx - fullwidth)
        if col > 0:
            add_neighbor(big_nodes, idx, idx - 1)
            
num_neighbors = 0
for node in big_nodes:
    num_neighbors += len(node.neighbors)
print(num_neighbors)
big_nodes[-1].end = True

250000
998000


In [127]:
from heapq import heappush, heappop
import itertools

# Based on the implementation https://docs.python.org/3/library/heapq.html
class PriorityQueue(object):
    
    def __init__(self):
        self.pq = []
        self.entry_finder = {}
        self.counter = itertools.count()

    def empty(self):
        return len(self.pq) == 0
            
    def get_cost(self, task):
        return self.entry_finder[task][0]
        
    def add_task(self, task, priority):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove_task(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def remove_task(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = None

    def pop_task(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.pq:
            priority, count, task = heappop(self.pq)
            if task is not None:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')


In [128]:
def solve(nodes):
    paths = []
    for _ in range(len(nodes)):
        paths.append(None)
    paths[0] = [nodes[0]]
    visited = [False] * len(nodes)

    cost = [9999999999] * len(nodes)
    unvisited = PriorityQueue()
    for node in nodes[1:]:
        unvisited.add_task(node, 9999999999)
    cur_node = nodes[0]
    cost[0] = 0
    visited[0] = True
    while not unvisited.empty():
        if cur_node.end:
            print(nodes[0].value)
            print(sum([node.value for node in paths[cur_node.idx]]))
            break
        for neighbor in cur_node.neighbors:
            if not visited[neighbor.idx]:
                potential_cost = cost[cur_node.idx] + neighbor.value
                if potential_cost < cost[neighbor.idx]:
                    paths[neighbor.idx] = paths[cur_node.idx] + [neighbor]
                    cost[neighbor.idx] = potential_cost
                    unvisited.add_task(neighbor, potential_cost)
        cur_node = unvisited.pop_task()
        visited[cur_node.idx] = True
        
solve(nodes)
solve(big_nodes)

2
388
2
2808
